In [1]:
# Replace this with pip install stabilizer
import sys
sys.path.insert(0, '..')

In [2]:
import torch
import pandas as pd

from stabilizer.dataset import TextLabelDataset

In [3]:
config = {'train_data_path': '../data/glue/cola/train.jsonl',
          'valid_data_path': '../data/glue/cola/valid.jsonl'}

In [4]:
# Read training data
train_data = pd.read_json(path_or_buf=config['train_data_path'], lines=True).set_index("idx")
valid_data = pd.read_json(path_or_buf=config['valid_data_path'], lines=True).set_index("idx")

In [5]:
# Prepare data to create dataset
train_text_excerpts = train_data['text'].tolist()
valid_text_excerpts = valid_data['text'].tolist()
train_labels = torch.from_numpy(train_data['label'].to_numpy().reshape(-1, 1)).type(torch.float32)
valid_labels = torch.from_numpy(valid_data['label'].to_numpy().reshape(-1, 1)).type(torch.float32)

In [6]:
# Create Dataset
train_dataset = TextLabelDataset(text_excerpts=train_text_excerpts, labels=train_labels)
valid_dataset = TextLabelDataset(text_excerpts=valid_text_excerpts, labels=valid_labels)